In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

from google.colab import drive
drive.mount('/content/drive')

import random
import cv2
from matplotlib import pyplot as plt
import albumentations as A
import pandas as pd
import os
import json

In [ ]:
#------------------ORIGINAL------------------------
# TRAIN_JSON = "/content/drive/My Drive/thyroid_nodule_detection/Experiment_3/dataset/load_json/train.json"
# TRAIN_DIR = "/content/drive/My Drive/thyroid_nodule_detection/dataset/train/"

# VALID_JSON = "/content/drive/My Drive/thyroid_nodule_detection/Experiment_3/dataset/load_json/validation.json"
# VALID_DIR = "/content/drive/My Drive/thyroid_nodule_detection/dataset/validation/"

# RESULT_JSON_FILE = "/content/drive/My Drive/thyroid_nodule_detection/Experiment_3/dataset/validation/resized_images.json"

# savepath = "/content/drive/My Drive/thyroid_nodule_detection/Experiment_3/dataset/validation/resized_images/"
#------------------ORIGINAL------------------------

#------------------FIXED LABELS------------------------
TRAIN_JSON = "/content/drive/My Drive/thyroid_nodule_detection/Freehand to Bounding Box Conversion Fix/fixed_labels/train.json"
TRAIN_DIR = "/content/drive/My Drive/thyroid_nodule_detection/dataset/train/"

VALID_JSON = "/content/drive/My Drive/thyroid_nodule_detection/Freehand to Bounding Box Conversion Fix/fixed_labels/validation.json"
VALID_DIR = "/content/drive/My Drive/thyroid_nodule_detection/dataset/validation/"

TEST_JSON = "/content/drive/My Drive/thyroid_nodule_detection/Freehand to Bounding Box Conversion Fix/fixed_labels/test.json"
TEST_DIR = "/content/drive/My Drive/thyroid_nodule_detection/dataset/test/"

ALL_DATA_JSON = "/content/drive/My Drive/thyroid_nodule_detection/Freehand to Bounding Box Conversion Fix/fixed_labels/all_data.json"
ALL_DATA_DIR = "/content/drive/My Drive/thyroid_nodule_detection/dataset/all_data/"

# Change paths to train/validation accordingly
RESULT_JSON_FILE = "/content/drive/My Drive/thyroid_nodule_detection/Freehand to Bounding Box Conversion Fix/fixed_labels/cropped_dataset/all_data/cropped_labels.json"
savepath = "/content/drive/My Drive/thyroid_nodule_detection/Freehand to Bounding Box Conversion Fix/fixed_labels/cropped_dataset/all_data/cropped_images/"
#------------------FIXED LABELS------------------------

In [ ]:
temp_train_df = pd.read_json(TRAIN_JSON)

In [ ]:
temp_valid_df = pd.read_json(VALID_JSON)

In [ ]:
temp_test_df = pd.read_json(TEST_JSON)

In [ ]:
temp_all_data_df =pd.read_json(ALL_DATA_JSON)

In [ ]:
BOX_COLOR = (255, 0, 0) # Red
TEXT_COLOR = (255, 255, 255) # White


def visualize_bbox(img, bbox, class_name, color=BOX_COLOR, thickness=2):
    """Visualizes a single bounding box on the image"""
    x_min, y_min, w, h = bbox
    x_min, x_max, y_min, y_max = int(x_min), int(x_min + w), int(y_min), int(y_min + h)
    # x_min, y_min, x_max, y_max = bbox
    # x_min, x_max, y_min, y_max = int(x_min), int(x_max), int(y_min), int(y_max)
   
    cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color=color, thickness=thickness)
    
    ((text_width, text_height), _) = cv2.getTextSize(class_name, cv2.FONT_HERSHEY_SIMPLEX, 0.35, 1)    
    cv2.rectangle(img, (x_min, y_min - int(1.3 * text_height)), (x_min + text_width, y_min), BOX_COLOR, -1)
    cv2.putText(
        img,
        text=class_name,
        org=(x_min, y_min - int(0.3 * text_height)),
        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
        fontScale=0.35, 
        color=TEXT_COLOR, 
        lineType=cv2.LINE_AA,
    )
    return img

def visualize(image, bboxes, category_ids, category_id_to_name):
    img = image.copy()
    for bbox, category_id in zip(bboxes, category_ids):
        class_name = category_id_to_name[category_id]
        img = visualize_bbox(img, bbox, class_name)
    plt.figure(figsize=(12, 12))
    plt.axis('off')
    plt.imshow(img)

In [ ]:
transform = A.Compose(
    [A.CenterCrop(height=350, width=400, p=1)],
    bbox_params=A.BboxParams(format='coco', min_area=1, label_fields=['category_ids']),
)

In [ ]:
def process_df(df):
    cases = df
    case_list = []
    for case in df['cases']:
        image_id = case['case_id'] + ".jpg"
        temp_case = {}
        
        temp_case['case_id'] = case['case_id']
        temp_case['image'] = cv2.imread(os.path.join(ALL_DATA_DIR, image_id), cv2.IMREAD_COLOR) # Change path to train/validation accordingly
        temp_case['tirads'] = case['tirads']
        temp_case['bboxes'] = case['bboxes']
        case_list.append(temp_case)
    return pd.DataFrame(case_list)

df_general = process_df(temp_all_data_df) # Change df to train/validation/test accordingly

In [ ]:
print(df_general)

In [ ]:
category_id_to_name = {2: 'benign', 3: 'benign',4: 'malignant',5: 'malignant' }

In [ ]:
def proccess_bboxes(bboxes):
  final_bboxes = []
  for bbox in bboxes:
    temp_bbox = [
        bbox['x'],
        bbox['y'],
        bbox['w'],
        bbox['h'],
    ]
    final_bboxes.append(temp_bbox)
  
  return final_bboxes

In [ ]:
def create_transormed_images(df):
  transformed_array = []
  for index,row in df.iterrows():
    id = row.get('tirads')
    if len(id) == 2:
      id = 4
    else:
      id = int(id)

    
    lenght_of_array = len(row.get('bboxes'))
    ## To add [id], lenght_of_array times, to category_ids:
    category_ids = lenght_of_array * [id]
    image = row.get('image')
    bboxes = proccess_bboxes(row.get('bboxes'))
    id = row.get('case_id')
    print(id)
    temp_transformed = transform(image=image, bboxes=bboxes, category_ids=category_ids)
    transormed_data = {
          'case_id': row.get('case_id'),
          'image': temp_transformed['image'],
          'bboxes': temp_transformed['bboxes'],
          'labels': temp_transformed['category_ids'],
      }
    transformed_array.append(transormed_data)
    cases = {
        'cases':transformed_array
    }
  return cases


usr/local/lib/python3.7/dist-packages/albumentations/core/bbox_utils.py/check_bbox

```
  bbox=list(bbox)
    for i in range(4):
      if bbox[i]<0:
        bbox[i]=0
      elif bbox[i]>1:
        bbox[i]=1
    bbox=tuple(bbox)
```


command + m + . = restart runtime

In [ ]:
transformed_data = create_transormed_images(df_general)
# print(transformed_data)

**SAVE IMAGES**

In [ ]:
import io
import PIL.Image as Image

for d in transformed_data['cases']:
  if len(d['bboxes']) >= 1:
    image_path = savepath + d['case_id'] + '.jpg'
    im = Image.fromarray(d['image'])
    im.save(image_path)

In [ ]:
temp_cases = []
for case in transformed_data['cases']:
  if len(case['bboxes']) >= 1:
    case = {
        'case_id':case['case_id'],
        'bboxes': case['bboxes'],
        'labels': case['labels']
    }
    temp_cases.append(case)

cases = {
    'cases': temp_cases
}
print(cases)

text_file = open(RESULT_JSON_FILE, "w")
text_file.write(json.dumps(cases))
text_file.close()

    